In [7]:
from rtk.task import KrakenAltoCleanUpCommand, YALTAiCommand, KrakenRecognizerCommand, ExtractZoneAltoCommand
from rtk import utils
import glob
from sys import argv
import time
import os

In [2]:
#folders = glob.glob("sample/*")

In [3]:
folders = glob.glob("sample/*.jpg")

In [4]:
start = time.time()

In [5]:
if len(argv) == 2:
    num_workers = int(argv[1])
else:
    num_workers = 5

In [9]:
for i in range(0, len(folders), 4):
    batch = folders[i:i+4]
    startYalt = time.time()
    startYalt = time.time()
    # Apply YALTAi
    print("[Task] Segment, size of batch ", len(batch))
    yaltai = YALTAiCommand(
        batch,
        binary="yaltai",
        device="cpu",
        yolo_model="models/ladas-1280-l.pt",
        verbose=True,
        raise_on_error=False,
        allow_failure=False,
        multiprocess=10,
        check_content=False,
        line_model = "models/blla.mlmodel"
    )
    yaltai.process()
    endYalt = time.time()
    
    print("[Time] Yaltai: ", endYalt - startYalt)
    print("Yaltai output files len ", len(yaltai.output_files))
    print("[Task] Fix ALTO file paths") 
    # Fix paths for current batch
    batch_xml_files = [img.replace('.jpg', '.xml') for img in batch]
    for alto_file in batch_xml_files:
        if os.path.exists(alto_file):
            with open(alto_file, 'r') as f:
                content = f.read()
            fixed_content = content.replace('<fileName>sample/', '<fileName>')
            with open(alto_file, 'w') as f:
                f.write(fixed_content)
    
    print("[Task] Clean-Up Serialization")
    cleanup = KrakenAltoCleanUpCommand(yaltai.output_files)
    cleanup.process()
    
    startKrak = time.time()
    print("[Task] OCR")
    kraken = KrakenRecognizerCommand(
        yaltai.output_files,
        binary="kraken",
        device="cpu",
        model="models/catmus-print-fondue-large.mlmodel",
        multiprocess=10,
        check_content=True
    )
    kraken.process()
    endKrak = time.time()
    
    print("[Time] Kraken: ", endKrak - startKrak)
end = time.time()
print("[Time] total: ", end - start)

[Task] Segment, size of batch  4


[Subtask] Processing YALTAi segmenter command: 100%|██████████| 4/4 [00:17<00:00,  4.31s/it]


[Time] Yaltai:  17.25054097175598
Yaltai output files len  4
[Task] Fix ALTO file paths
[Task] Clean-Up Serialization


[Subtask] Checking prior processed documents: 100%|██████████| 4/4 [00:00<00:00, 2886.15it/s]


Nothing to process here.
[Task] OCR


...

INPUT SIZE DIFF WITH OUTPUT SIZE
['', 'image 1/1 /Users/carboni/Documents/pynotebook/personal/12.01 Iconographical Canon/Processing/sample/29432253.jpg: 1280x768 1 MainZone-Continued, 1 MainZone-Head, 2 MainZone-Sps, 4 MarginTextZone-Notess, 2 NumberingZones, 1 QuireMarksZone, 1 RunningTitleZone, 757.4ms', 'Speed: 6.9ms preprocess, 757.4ms inference, 0.5ms postprocess per image at shape (1, 3, 1280, 768)', '\x1b[09/08/25 11:41:04]\x1b\x1b \x1b\x1bINFO    \x1b Vectorizing baselines                                                                                                                                                                                                                                               \x1b]8;id=884825;file:///opt/homebrew/Caskroom/miniforge/base/envs/icon/lib/python3.10/site-packages/kraken/blla.py\x1b\\\x1bblla.py\x1b\x1b]8;;\x1b\\\x1b:\x1b\x1b]8;id=558109;file:///opt/homebrew/Caskroom/miniforge/base/envs/icon/lib/python3.10/site-packages/kraken/blla.py#2

[Subtask] Checking prior processed documents: 0it [00:00, ?it/s]


Nothing to process here.
[Task] OCR


[Subtask] Checking prior processed documents: 0it [00:00, ?it/s]

Nothing to process here.
[Time] Kraken:  0.0007390975952148438
[Time] total:  143.96246814727783
